# Statistical Downscaling and Bias-Adjustment - Advanced tools

The previous notebook covered the most common utilities of `xclim.sdba` for conventionnal cases. Here we explore more advanced usage of `xclim.sdba` tools.

## LOESS smoothing and detrending

As described in Cleveland (1979), locally weighted linear regressions are multiple regression methods using a nearest-neighbor approach. Instead of using all data points to compute a linear or polynomial regression, LOESS algorithms compute a local regression for each point in the dataset, using only the k-nearest neighbors as selected by a weighting function. This weighting function must fulfill some strict requirements, see the doc of `xclim.sdba.loess.loess_smoothing` for more details.

In xclim's implementation, the user can choose between local _constancy_ ($d=0$, local estimates are weighted averages) and local _linearity_ ($d=1$, local estimates are taken from linear regressions). Two weighting functions are currently implemented : "tricube" ($w(x) = (1 - x^3)^3$) and "gaussian" ($w(x) = e^{-x^2 / 2\sigma^2}$). Finally, the number of Cleveland's _robustifying iterations_ is controllable through `niter`. After computing an estimate of $y(x)$, the weights are modulated by a function of the distance between the estimate and the points and the procedure is started over. These iterations are made to weaken the effect of outliers on the estimate.

The next example shows the application of the LOESS to daily temperature data. The black line and dot are the estimated $y$, outputs of the `sdba.loess.loess_smoothing` function, using local linear regression (passing $d = 1$), a window spanning 20% ($f = 0.2$) of the domain, the "tricube" weighting function and only one iteration. The red curve illustrates the weighting function on January 1st 2014, where the red circles are the nearest-neighbors used in the estimation.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from xclim.sdba import loess
%matplotlib inline

In [ ]:
# Daily temperature data from xarray's tutorials
ds = xr.tutorial.open_dataset('air_temperature').resample(time='D').mean()
tas = ds.isel(lat=0, lon=0).air

# Compute the smoothed series
f = 0.2
ys = loess.loess_smoothing(tas, d=1, weights='tricube', f=f, niter=1)

# Plot data points and smoothed series
fig, ax = plt.subplots()
ax.plot(tas.time, tas, 'o', fillstyle='none')
ax.plot(tas.time, ys, 'k')
ax.set_xlabel('Time')
ax.set_ylabel('Temperature [K]')

## The code below calls internal functions to demonstrate how the weights are computed. 

# LOESS algorithms as implemented here use scaled coordinates.
x = tas.time
x = (x - x[0]) / (x[-1] - x[0])
xi = x[366]
ti = tas.time[366]

# Weighting function take the distance with all neighbors scaled by the r parameter as input
r = int(f * tas.time.size)
h = np.sort(np.abs(x - xi))[r]
weights = loess._tricube_weighting(np.abs(x - xi).values / h)

# Plot nearest neighbors and weighing function
wax = ax.twinx()
wax.plot(tas.time, weights, color='indianred')
ax.plot(tas.time, tas.where(tas * weights > 0), 'o', color='lightcoral', fillstyle='none')

ax.plot(ti, ys[366], 'ko')
wax.set_ylabel('Weights')
plt.show()

LOESS smoothing can suffer from heavy boundary effects. On the previous graph, we can associate the strange bend on the left end of the line to them. The next example shows a stronger case. Usually, $\frac{f}{2}N$ points on each side should be discarded. On the other hand, LOESS has the advantage of always staying within the bounds of the data.


### LOESS Detrending

In climate science, it can be used in the detrending process. `xclim` provides `sdba.detrending.LoessDetrend` in order to compute trend with the LOESS smoothing and remove them from timeseries.

First we create some toy data with a sinusoidal annual cycle, random noise and a linear temperature increase.

In [ ]:
time = xr.cftime_range('1990-01-01', '2049-12-31', calendar='noleap')
tas = xr.DataArray(
   (10 * np.sin(time.dayofyear * 2 * np.pi / 365) +  # Annual variability
    5 * (np.random.random_sample(time.size) - 0.5) +  # Random noise
    np.linspace(0, 1.5, num=time.size)),  # 1.5 degC increase in 60 years
    dims=('time',), coords={'time': time},
    attrs={'units': 'degC'}, name='temperature',
)
tas.plot()

Then we compute the trend on the data. Here, we compute on the whole timeseries (`group='time'`) with the parameters suggested above.

In [ ]:
from xclim.sdba.detrending import LoessDetrend

# Create the detrending object
det = LoessDetrend(group='time', d=0, niter=2, f=0.2)
# Fitting returns a new object
fit = det.fit(tas)
# Get the trend and the detrended series
trend = fit.get_trend(tas)
tas_det = fit.detrend(tas)

In [ ]:
fig, ax = plt.subplots()
trend.plot(ax=ax, label='Computed trend')
ax.plot(time, np.linspace(0, 1.5, num=time.size), label='Expected tred')
ax.plot([time[0], time[int(0.1 * time.size)]], [0.4, 0.4], linewidth=6, color="gray")
ax.plot([time[-int(0.1 * time.size)], time[-1]], [1.1, 1.1], linewidth=6, color="gray")
ax.legend()

As said earlier, this example shows how the Loess has strong boundary effects. It is recommended to remove the $\frac{f}{2}\cdot N$ outermost points on each side, as shown by the  gray bars in the graph above.

## Initializing an Adjustment object from a training dataset

For large scale uses, when the training step deserves its own computation and write to disk, or simply when there are multiples `sim` to be adjusted with the same training, it is helpful to be able to instantiate the Adjustment objects from the training dataset itself. This trick relies on a global attribute "adj_params" set on the training dataset.

In [ ]:
import numpy as np
import xarray as xr

# Create toy data for the example, here fake temperature timeseries
t = xr.cftime_range('2000-01-01', '2030-12-31', freq='D', calendar='noleap')
ref = xr.DataArray((-20 * np.cos(2 * np.pi * t.dayofyear / 365) + 2 * np.random.random_sample((t.size,)) + 273.15
                    + 0.1 * (t - t[0]).days / 365),  # "warming" of 1K per decade,
                   dims=('time',), coords={'time': t}, attrs={'units': 'K'})
sim = xr.DataArray((-18 * np.cos(2 * np.pi * t.dayofyear / 365) + 2 * np.random.random_sample((t.size,)) + 273.15
                    + 0.11 * (t - t[0]).days / 365),  # "warming" of 1.1K per decade
                   dims=('time',), coords={'time': t}, attrs={'units': 'K'})

ref = ref.sel(time=slice(None, '2015-01-01'))
hist = sim.sel(time=slice(None, '2015-01-01'))

In [ ]:
from xclim.sdba.adjustment import QuantileDeltaMapping

QDM = QuantileDeltaMapping(nquantiles=15, kind='+', group='time.dayofyear')
QDM.train(ref, hist)
QDM

The trained `QDM` exposes the training data in the `ds` attribute, Here, we will write it to disk, read it back and initialize an new object from it. Notice the `adj_params` in the dataset, that has the same value as the repr string printed just above. Also, notice the `_xclim_adjustment` attribute that contains a json string so we can rebuild the adjustment object later.

In [ ]:
QDM.ds

In [ ]:
QDM.ds.to_netcdf('QDM_training.nc')
ds = xr.open_dataset('QDM_training.nc')
QDM2 = QuantileDeltaMapping.from_dataset(ds)
QDM2

In the case above, creating a full object from the dataset doesn't make the most sense since we are in the same python session, with the "old" object still available. This method effective when we reload the training data in a different python session, say on another computer. **However, take note that there is no retrocompatiblity insurance.** If the QuantileDeltaMapping class was to change in a new xclim version, one would not be able to create the new object from a dataset saved with the old one.

For the case where we stay in the same python session, it is still useful to trigger the dask computations. For small datasets, that could mean a simple `QDM.ds.load()`, but sometimes even the training data is too large to be full loaded in memory. In that case, we could also do:

In [ ]:
QDM.ds.to_netcdf('QDM_training2.nc')
ds = xr.open_dataset('QDM_training2.nc')
ds.attrs['title'] = 'This is the dataset, but read from disk.'
QDM.set_dataset(ds)
QDM.ds

In [ ]:
QDM2.adjust(sim)